#Image Preprocessing

In [5]:
#Importing library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
#Image Augumentation
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [20]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vUJQ2E5Sw27HXjm6dTeMQbgSpVJzyUt9Jl-w6gnTi605',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-wlfrq46lfjtwyq'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [21]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [23]:
#Loading train set and test set
X_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/training_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
X_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/test_set", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [24]:
#Checking indices
X_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

#Model Building

In [25]:
#Importing libraries

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

In [26]:
#Initialize the Model
model = Sequential()

In [27]:
#Adding Convolution Layer
model.add(Convolution2D((32), (3,3), input_shape = (64, 64, 3), activation = 'relu'))

In [28]:
#Adding Pooling Layer
model.add(MaxPooling2D(pool_size = (2, 2)))

In [29]:
#Adding Flatten Layer
model.add(Flatten())

In [30]:
#Adding dense layers
model.add(Dense(units = 512, kernel_initializer = 'random_uniform', activation = 'relu'))

In [31]:
model.add(Dense(units = 9, kernel_initializer = 'random_uniform', activation = 'softmax'))

In [32]:
#Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [28]:
#Fiiting the model
model.fit_generator(X_train, steps_per_epoch = 24, epochs = 10, validation_data = X_test, validation_steps = 40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
24/24 [==============================] - 13s 480ms/step - loss: 1.8073 - accuracy: 0.4818 - val_loss: 0.8104 - val_accuracy: 0.6992
Epoch 2/10
24/24 [==============================] - 9s 369ms/step - loss: 0.6137 - accuracy: 0.8021 - val_loss: 0.4225 - val_accuracy: 0.8805
Epoch 3/10
24/24 [==============================] - 9s 379ms/step - loss: 0.3315 - accuracy: 0.9102 - val_loss: 0.3937 - val_accuracy: 0.8898
Epoch 4/10
24/24 [==============================] - 9s 372ms/step - loss: 0.2432 - accuracy: 0.9232 - val_loss: 0.3050 - val_accuracy: 0.9141
Epoch 5/10
24/24 [==============================] - 9s 374ms/step - loss: 0.1876 - accuracy: 0.9479 - val_loss: 0.3633 - val_accuracy: 0.9094
Epoch 6/10
24/24 [==============================] - 9s 382ms/step - loss: 0.2168 - accuracy: 0.9375 - val_loss: 0.3985 - val_accuracy: 0.8766
Epoch 7/10
24/24 [==============================] - 9s 368ms/step - loss: 0.1614 - accuracy: 0.9505 - val_loss: 0.2754 - val_accuracy: 0.9414
Epoch

In [33]:
#Saving the model
model.save('aslpng1.h5')

In [36]:
!tar -zcvf image-classification-model.tgz aslpng1.h5

aslpng1.h5


In [35]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 10.5 MB/s eta 0:00:01


In [37]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "4K5Qo9nCXKGDes73rlgGVGMsTCaEXaINd8FWziAiYHf_"
}
client = APIClient(wml_credentials)

In [38]:
def guid_space_name(client, ai_based_real_time_communication_deploy_space):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == ai_based_real_time_communication_deploy_space)['metadata']['id'])

In [39]:
client.spaces.get_details()

{'resources': [{'entity': {'description': '',
    'name': 'imageclassification',
    'scope': {'bss_account_id': 'b1fd66c9342e40e18cd868ebc899f045'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': 'd032cf9e-bf0c-47bd-b573-0fa4f13976a9',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '9a6490f9f27b42b0b35949afd235b565',
        'api_key': 'o2ZFO4bKCF_ZMkku1tgpiCPWOzNG3Fh1o3ziXPvaq0v1',
        'secret_access_key': '4311fd0cc79863a20cac6e748ce55ea7110c010fe265c5c8',
        'service_id': 'ServiceId-1243d1a6-3478-4301-8612-cfc37c773e3e'},
       'editor': {'access_key_id': 'bc31dc1898964d25acab41ffdfe61c7b',
        'api_key': '7noezH3x6OsJ635QywKcr2zxX69DCsDXe9WF1SvA2LcK',
        'resource_key_crn': 'crn:v1:bluemix:public:cloud-object-storage:global:a/b1fd66c9342e40e18cd868ebc899f045:58aadd6e-354d-4128-92dd-88336e29c8fa::',
        'secret_access_key': '5fcde3dbda2aec00df179213f1804

In [40]:
space_id = guid_space_name(client, 'imageclassification')
space_id

'6a8a9254-afb7-498e-9c5f-63f74b13a680'

In [41]:
client.set.default_space(space_id)

'SUCCESS'

In [42]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [43]:
software_space_id = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model = 'image-classification-model.tgz', meta_props = {
    client.repository.ModelMetaNames.NAME:"CNN Model Buiding",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id
})

In [46]:
model_id = client.repository.get_model_id(model_details)
model_id

'5c62850b-2828-455e-b1c5-eb6e1b034217'

In [47]:
client.repository.download(model_id, 'image-classification-model.h5')

Successfully saved model content to file: 'image-classification-model.h5'


'/home/wsuser/work/image-classification-model.h5'